# Nasdaq 100 与 S&P 500 历年年度涨幅对比

本 Notebook 使用 Yahoo Finance 数据（通过 `yfinance` 库）抓取 **Nasdaq 100** 与 **S&P 500** 指数，
并按自然年度计算各年的价格涨跌幅（不含分红再投资），用于长期表现对比。

> 提示：运行前请确保已安装 `yfinance`，并在有网络的环境中运行。


In [1]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['axes.unicode_minus'] = False

pd.__version__

'2.3.2'

## 1. 设置指数与时间区间

- Nasdaq 100 指数代码：`^NDX`
- S&P 500 指数代码：`^GSPC`

这里从 1985 年开始抓取数据，你可以根据需要调整起始日期。

In [2]:
INDEXES = {
    'Nasdaq 100': '^NDX',
    'S&P 500': '^GSPC',
}

start = '1985-01-01'   # 起始日期，可自行调整
end = None             # 使用最新可用数据


## 2. 抓取日度收盘价

使用 `Adj Close`（复权收盘价）来计算年度涨跌幅。

In [3]:
all_prices = []

for name, ticker in INDEXES.items():
    print(f'Fetching {name} ({ticker}) ...')
    data = yf.download(ticker, start=start, end=end, progress=False)
    if data.empty:
        raise ValueError(f'No data downloaded for {name} / {ticker}.')
    s = data['Adj Close'].rename(name)
    all_prices.append(s)

prices = pd.concat(all_prices, axis=1).dropna(how='all')
prices.tail()

Fetching Nasdaq 100 (^NDX) ...


/var/folders/f5/77dy_6fs23x_7_g_4f_zfdn00000gn/T/ipykernel_29713/290101479.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start, end=end, progress=False)

1 Failed download:
['^NDX']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


ValueError: No data downloaded for Nasdaq 100 / ^NDX.

## 3. 计算自然年度涨跌幅

方法：
1. 将日度价格按年重采样到 **每年最后一个交易日** 的收盘价。
2. 对重采样后的序列做 `pct_change()`，得到年度涨跌幅。
3. 乘以 100 得到百分比形式，并做适当四舍五入。

In [ ]:
# 年末价格（每年的最后一个交易日）
year_end_prices = prices.resample('Y').last()

# 计算年度涨跌幅（不含分红再投资）
annual_returns = year_end_prices.pct_change() * 100
annual_returns = annual_returns.dropna(how='all')
annual_returns = annual_returns.round(2)

# 将 index 改为自然年数字
annual_returns.index = annual_returns.index.year
annual_returns

## 4. 美化表格展示

将年度涨跌幅格式化为百分号形式，便于直观读取。

若你在导出到 Excel 时遇到 `Styler` 相关问题，可以直接使用 `annual_returns.to_excel(...)` 导出，
不依赖样式。

In [ ]:
def fmt_pct(x: float) -> str:
    return f'{x:.2f}%' if pd.notna(x) else ''

styled = annual_returns.style.format(fmt_pct)
styled

## 5. 绘制年度涨跌幅柱状图

简单对比 Nasdaq 100 与 S&P 500 各年的涨跌幅情况。

In [ ]:
ax = annual_returns.plot(kind='bar')
ax.set_title('Nasdaq 100 vs S&P 500 年度涨跌幅')
ax.set_ylabel('年度涨跌幅 (%)')
ax.legend(loc='best')
plt.tight_layout()
plt.show()

## 6. 可选：导出为 CSV / Excel

如果你需要在 Excel 里进一步处理，可以直接导出。

In [ ]:
# annual_returns.to_csv('nasdaq_sp500_annual_returns.csv', encoding='utf-8-sig')
# annual_returns.to_excel('nasdaq_sp500_annual_returns.xlsx')

annual_returns.head()